## Data Set Construction

**Functions**

`pd.read_csv`, `pd.read_excel`, `np.diff` or `DataFrame.diff`, `DataFrame.resample`

### Exercise 1

1. Download all available daily data for the S&P 500 and the Hang Seng Index from Yahoo! Finance. 
2. Import both data sets into Python. The final dataset should have a `DateTimeIndex`, and the date
   column should not be part of the `DataFrame`.
3. Construct weekly price series from each, using Tuesday prices (less likely to be a holiday).
4. Construct monthly price series from each using last day in the month.
5. Save the data to the HDF file "equity-indices.h5".


In [1]:
import pandas as pd

sp500 = pd.read_csv("data/GSPC.csv", parse_dates=True, index_col="Date")
hsi = pd.read_csv("data/HSI.csv", parse_dates=True, index_col="Date")

weekly_sp500 = sp500.resample("W-TUE").last()
weekly_hsi = hsi.resample("W-TUE").last()

monthly_sp500 = sp500.resample("M").last()
monthly_hsi = hsi.resample("M").last()

h5file = pd.HDFStore("data/equity-indices.h5", mode="w")
h5file.append("sp500", sp500)
h5file.append("weekly_sp500", weekly_sp500)
h5file.append("monthly_sp500", monthly_sp500)
h5file.append("hsi", sp500)
h5file.append("weekly_hsi", weekly_hsi)
h5file.append("monthly_hsi", monthly_hsi)
h5file.close()

sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-09-17,2995.669922,3006.209961,2993.729980,3005.699951,3005.699951,3671840000
2019-09-18,3001.500000,3007.830078,2978.570068,3006.729980,3006.729980,3435540000
2019-09-19,3010.360107,3021.989990,3003.159912,3006.790039,3006.790039,3251290000
2019-09-20,3008.419922,3016.370117,2984.679932,2992.070068,2992.070068,6094740000
2019-09-23,2983.500000,2999.149902,2982.229980,2991.780029,2991.780029,3186590000


In [2]:
weekly_sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-08-27,2893.139893,2898.790039,2860.590088,2869.159912,2869.159912,3533630000
2019-09-03,2909.010010,2914.389893,2891.850098,2906.270020,2906.270020,3426790000
2019-09-10,2971.010010,2979.389893,2957.010010,2979.389893,2979.389893,4390770000
2019-09-17,2995.669922,3006.209961,2993.729980,3005.699951,3005.699951,3671840000
2019-09-24,2983.500000,2999.149902,2982.229980,2991.780029,2991.780029,3186590000


In [3]:
monthly_sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-05-31,2766.149902,2768.979980,2750.520020,2752.060059,2752.060059,3981020000
2019-06-30,2932.939941,2943.979980,2929.050049,2941.760010,2941.760010,5420700000
2019-07-31,3016.219971,3017.399902,2958.080078,2980.379883,2980.379883,4623430000
2019-08-31,2937.090088,2940.429932,2913.320068,2926.459961,2926.459961,3008450000
2019-09-30,2983.500000,2999.149902,2982.229980,2991.780029,2991.780029,3186590000


### Exercise 2

Write a function that will correctly aggregate to weekly or monthly respecting the
aggregation rules

* High: `max`
* Low: `min`
* Volume: `sum`

The signature should be:

```python
def yahoo_agg(data, freq):
    <code here>
    return resampled_data
```


In [4]:
def yahoo_agg(data, freq):
    resampler = data.resample(freq)

    high = resampler.High.max()
    low = resampler.Low.min()
    vol = resampler.Volume.sum()
    # Start with last for all columns
    resampled_data = resampler.last()
    # Insert columns that use a different rule
    resampled_data["High"] = high
    resampled_data["Low"] = low
    resampled_data["Volume"] = vol

    return resampled_data


better_monthly_sp500 = yahoo_agg(sp500, "M")

monthly_sp500[["High", "Low", "Volume"]].tail()

,High,Low,Volume
Date,,,
2019-05-31,2768.979980,2750.520020,3981020000
2019-06-30,2943.979980,2929.050049,5420700000
2019-07-31,3017.399902,2958.080078,4623430000
2019-08-31,2940.429932,2913.320068,3008450000
2019-09-30,2999.149902,2982.229980,3186590000


In [5]:
better_monthly_sp500[["High", "Low", "Volume"]].tail()

,High,Low,Volume
Date,,,
2019-05-31,2954.129883,2750.520020,76860120000
2019-06-30,2964.149902,2728.810059,70881390000
2019-07-31,3027.979980,2952.219971,70349470000
2019-08-31,3013.590088,2822.120117,79599440000
2019-09-30,3021.989990,2891.850098,57236800000


### Exercise 3

1. Import the Fama-French benchmark portfolios as well as the 25 sorted portfolios at both the
   monthly and daily horizon from [Ken French"s Data Library](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html).
   **Note** It is much easier to clean to data file before importing than to find the precise
   command that will load the unmodified data.
2. Import daily FX rate data for USD against AUD, Euro, JPY and GBP from the [Federal Reserve Economic Database (FRED)](http://research.stlouisfed.org/fred2/categories/94). Use Excel (xls) rather than csv files.
3. Save the data to the HDF files "fama-french.h5" and "fx.h5"

In [6]:
yen_dollar = pd.read_excel(
    "data/DEXJPUS.xls", index_col="observation_date", skiprows=10
)
dollar_aud = pd.read_excel(
    "data/DEXUSAL.xls", index_col="observation_date", skiprows=10
)
dollar_euro = pd.read_excel(
    "data/DEXUSEU.xls", index_col="observation_date", skiprows=10
)
dollar_pound = pd.read_excel(
    "data/DEXUSUK.xls", index_col="observation_date", skiprows=10
)

fx = pd.concat([yen_dollar, dollar_aud, dollar_euro, dollar_pound], axis=1)
print(fx.tail())
fx.to_hdf("data/fx.h5", "fx")

                  DEXJPUS  DEXUSAL  DEXUSEU  DEXUSUK
observation_date                                    
2019-09-16         107.94   0.6865   1.1006   1.2423
2019-09-17         108.12   0.6858   1.1063   1.2493
2019-09-18         108.17   0.6844   1.1061   1.2478
2019-09-19         108.00   0.6795   1.1046   1.2481
2019-09-20         107.95   0.6770   1.1004   1.2489


In [7]:
# These files have all been cleaned to have only the data and headers
ff_5x5 = pd.read_csv("data/25_Portfolios_5x5.CSV", index_col=0)
ff_factors = pd.read_csv("data/F-F_Research_Data_Factors.CSV", index_col=0)
ff = pd.concat([ff_factors, ff_5x5], axis=1)

dates = []
for value in ff.index:
    # Values are YYYYMM
    year = value // 100
    month = value % 100
    dates.append(pd.Timestamp(year=year, month=month, day=1))
ff.index = dates
ff.tail()

,Mkt-RF,SMB,HML,RF,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
2019-03-01,1.10,-3.13,-4.07,0.19,0.8891,-0.4743,-2.5100,-4.9485,-3.4235,-0.2319,...,1.8237,1.0998,-0.9002,-0.5395,-2.8702,3.3344,1.9283,-0.1602,-0.5929,-3.0620
2019-04-01,3.96,-1.68,1.93,0.21,0.6068,1.6518,2.1840,3.4912,2.0865,2.8361,...,3.1410,4.2046,4.3501,5.0762,4.3419,4.6039,2.9807,3.6429,5.1341,9.3863
2019-05-01,-6.94,-1.20,-2.39,0.21,-5.5586,-5.3126,-7.7829,-8.8025,-7.9767,-8.9190,...,-5.7334,-7.1667,-7.7153,-9.1637,-7.9580,-6.9663,-5.7641,-5.0654,-8.1314,-8.7586
2019-06-01,6.93,0.33,-1.08,0.18,7.6353,6.2398,4.7677,6.7187,4.7008,8.9242,...,7.2875,9.4043,9.5103,7.5115,6.9246,7.7424,5.8872,5.9640,6.6670,7.9347
2019-07-01,1.19,-2.06,0.12,0.19,-3.8504,-1.1600,-0.8926,-1.6496,-2.5149,-3.1247,...,1.6768,1.0410,1.9332,-0.6625,0.9940,1.6565,1.9146,1.5096,0.9860,2.1159


In [8]:
# This is a "trick" to get the index to have the last day in the month.
ff = ff.resample("M").last()

ff.to_hdf("data/ff.h5", "ff")

ff.tail()

,Mkt-RF,SMB,HML,RF,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
2019-03-31,1.10,-3.13,-4.07,0.19,0.8891,-0.4743,-2.5100,-4.9485,-3.4235,-0.2319,...,1.8237,1.0998,-0.9002,-0.5395,-2.8702,3.3344,1.9283,-0.1602,-0.5929,-3.0620
2019-04-30,3.96,-1.68,1.93,0.21,0.6068,1.6518,2.1840,3.4912,2.0865,2.8361,...,3.1410,4.2046,4.3501,5.0762,4.3419,4.6039,2.9807,3.6429,5.1341,9.3863
2019-05-31,-6.94,-1.20,-2.39,0.21,-5.5586,-5.3126,-7.7829,-8.8025,-7.9767,-8.9190,...,-5.7334,-7.1667,-7.7153,-9.1637,-7.9580,-6.9663,-5.7641,-5.0654,-8.1314,-8.7586
2019-06-30,6.93,0.33,-1.08,0.18,7.6353,6.2398,4.7677,6.7187,4.7008,8.9242,...,7.2875,9.4043,9.5103,7.5115,6.9246,7.7424,5.8872,5.9640,6.6670,7.9347
2019-07-31,1.19,-2.06,0.12,0.19,-3.8504,-1.1600,-0.8926,-1.6496,-2.5149,-3.1247,...,1.6768,1.0410,1.9332,-0.6625,0.9940,1.6565,1.9146,1.5096,0.9860,2.1159


In [9]:
ff.index

DatetimeIndex(['1926-07-31', '1926-08-31', '1926-09-30', '1926-10-31',
               '1926-11-30', '1926-12-31', '1927-01-31', '1927-02-28',
               '1927-03-31', '1927-04-30',
               ...
               '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31',
               '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31',
               '2019-06-30', '2019-07-31'],
              dtype='datetime64[ns]', length=1117, freq='M')

In [10]:
# These files have all been cleaned to have only the data and headers
ff_5x5_daily = pd.read_csv("data/25_Portfolios_5x5_daily.CSV", index_col=0)
ff_factors_daily = pd.read_csv("data/F-F_Research_Data_Factors_daily.CSV", index_col=0)
ff_daily = pd.concat([ff_factors_daily, ff_5x5_daily], axis=1)


dates = []
for value in ff_daily.index:
    # Values are YYYYMMDD
    year = value // 10000
    month = (value // 100) % 100
    day = value % 100
    dates.append(pd.Timestamp(year=year, month=month, day=day))
ff_daily.index = dates
ff_daily.to_hdf("data/ff_daily.h5", "ff_daily")

ff_daily.tail()

,Mkt-RF,SMB,HML,RF,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
2019-07-25,-0.63,-0.69,-0.21,0.009,-1.89,-1.59,-1.02,-1.11,-1.59,-1.90,...,-0.95,-0.72,-0.74,-0.34,-1.23,-0.62,-0.27,-0.83,-0.48,-1.68
2019-07-26,0.82,0.39,-0.21,0.009,1.53,1.48,1.29,1.18,0.68,1.37,...,0.84,0.65,0.96,0.17,0.27,0.53,1.54,0.61,0.62,1.37
2019-07-29,-0.32,-0.32,-0.47,0.009,-0.34,0.16,-0.89,-0.62,-0.99,-0.82,...,-0.93,-0.36,-0.24,-0.86,-0.85,-0.05,-0.34,-0.32,-0.33,-0.91
2019-07-30,-0.17,1.31,0.86,0.009,1.04,1.49,0.84,1.21,2.13,0.86,...,-0.22,-0.18,0.41,0.95,1.25,-0.27,-0.70,-0.10,0.16,-0.48
2019-07-31,-1.09,0.13,0.56,0.009,-0.90,-1.76,-0.43,-0.73,-0.43,-1.44,...,-0.98,-1.30,-0.68,-0.65,-1.32,-1.24,-1.23,-1.24,-0.65,-0.64


### Exercise 3 (Alternative method)

1. Install and use `pandas-datareader` to repeat the previous exercise.

#### Preliminary Step

You must first install the module using 

```
pip install pandas-datareader
``` 

from the command line. Then you can run this code. **Note**: Running this code requires access
to the internet.

In [11]:
import pandas_datareader as pdr

# Conservative start date to get all data
yen_dollar = pdr.get_data_fred("DEXJPUS", start="1950")
dollar_aud = pdr.get_data_fred("DEXUSAL", start="1950")
dollar_euro = pdr.get_data_fred("DEXUSEU", start="1950")
dollar_pound = pdr.get_data_fred("DEXUSUK", start="1950")
fx = pd.concat([yen_dollar, dollar_aud, dollar_euro, dollar_pound], axis=1)
fx.to_hdf("data/fx-pdr.h5", "fx")
fx.tail()

,DEXJPUS,DEXUSAL,DEXUSEU,DEXUSUK
DATE,,,,
2024-02-19,NaN,NaN,NaN,NaN
2024-02-20,149.87,0.6560,1.0814,1.2645
2024-02-21,150.29,0.6546,1.0818,1.2624
2024-02-22,150.62,0.6547,1.0815,1.2649
2024-02-23,150.36,0.6566,1.0828,1.2678


In [12]:
ff_factors = pdr.get_data_famafrench("F-F_Research_Data_Factors", start="1920")
ff_5x5 = pdr.get_data_famafrench("25_Portfolios_5x5", start="1920")
# The function returns all of the tables in each file.  We want the first, [0]
ff_factors = ff_factors[0]
ff_5x5 = ff_5x5[0]
ff = pd.concat([ff_factors, ff_5x5], axis=1)
ff.to_hdf("data/ff-pdr.h5", "ff")
ff.tail()

C:\Users\kevin\AppData\Local\Temp\ipykernel_6172\127267269.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = pdr.get_data_famafrench("F-F_Research_Data_Factors", start="1920")
C:\Users\kevin\AppData\Local\Temp\ipykernel_6172\127267269.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = pdr.get_data_famafrench("F-F_Research_Data_Factors", start="1920")
C:\Users\kevin\AppData\Local\Temp\ipykernel_6172\127267269.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_5x5 = pdr.get_data_famafrench("25_Portf

,Mkt-RF,SMB,HML,RF,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
Date,,,,,,,,,,,,,,,,,,,,,
2023-08,-2.39,-3.16,-1.06,0.45,-12.2376,-7.6627,-10.5103,-5.6405,-7.3431,-6.9345,...,-2.4000,-2.3534,-2.8719,-3.8686,-2.1140,-0.8338,-0.9250,-2.6918,-2.9058,-7.3853
2023-09,-5.24,-2.51,1.52,0.43,-8.2275,-7.6201,-6.5856,-5.8185,-6.0216,-8.8904,...,-5.6917,-4.6306,-6.5792,-5.0335,-3.4990,-5.5603,-4.3422,-1.4851,-2.1397,-2.2370
2023-10,-3.19,-3.87,0.19,0.47,-10.2240,-8.9443,-7.9196,-6.3774,-7.6783,-10.2040,...,-5.9518,-5.8296,-7.1202,-5.7789,-6.2824,-0.9824,-4.9262,-2.5643,-2.6864,-4.0377
2023-11,8.84,-0.02,1.64,0.44,5.7861,8.0737,10.7319,8.5040,7.0945,10.9883,...,9.8743,8.9790,9.5279,8.7238,7.8476,9.8946,7.0176,7.0935,8.6614,15.2510
2023-12,4.85,6.35,4.94,0.43,15.3216,16.0383,14.8635,16.4605,14.9182,12.0839,...,8.6568,7.7652,9.1821,9.0094,11.7210,3.7289,4.7129,5.0841,4.6592,11.6076


### Exercise 4
Download data on 1 year and 10 year US government bond rates from FRED, and 
construct the term premium as the different in yields on 10 year and 1 year
bonds. Combine the two yield series and the term premium into a `DataFrame`
and save it as HDF.

In [13]:
# No need to import here since pandas and pandas-datareader previously imported

# Conservative start date to get all data
gs10 = pdr.get_data_fred("GS10", start="1950")
gs1 = pdr.get_data_fred("GS1", start="1950")

term = gs10["GS10"] - gs1["GS1"]
term.name = "TERM"
combined = pd.DataFrame([term, gs10["GS10"], gs1["GS1"]]).T
combined.tail()

,TERM,GS10,GS1
DATE,,,
2023-09-01,-1.06,4.38,5.44
2023-10-01,-0.62,4.80,5.42
2023-11-01,-0.78,4.50,5.28
2023-12-01,-0.94,4.02,4.96
2024-01-01,-0.73,4.06,4.79


In [14]:
combined.index

DatetimeIndex(['1953-04-01', '1953-05-01', '1953-06-01', '1953-07-01',
               '1953-08-01', '1953-09-01', '1953-10-01', '1953-11-01',
               '1953-12-01', '1954-01-01',
               ...
               '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01',
               '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01',
               '2023-12-01', '2024-01-01'],
              dtype='datetime64[ns]', name='DATE', length=850, freq=None)

In [15]:
# Trick to ensure the index has the frequency MS, Month Start
combined = combined.resample("MS").last()
combined.to_hdf("data/term-premium.h5", "term_premium")
combined.index

DatetimeIndex(['1953-04-01', '1953-05-01', '1953-06-01', '1953-07-01',
               '1953-08-01', '1953-09-01', '1953-10-01', '1953-11-01',
               '1953-12-01', '1954-01-01',
               ...
               '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01',
               '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01',
               '2023-12-01', '2024-01-01'],
              dtype='datetime64[ns]', name='DATE', length=850, freq='MS')